In [ ]:
# import settings and functions
%run ./../../imports.ipynb


## What mesh?

Copy your choice to the next cell

for SquareTop:
```
analytical_solution_tag = "-ana_square_top"
generate_config = generateConfig_squareTop
generate_mesh = generateMesh_squareTop
```

for SquareSinCos:
```
analytical_solution_tag = "-ana_square_sincos"
generate_config = generateConfig_squareSinCos
generate_mesh = generateMesh_squareSinCos
```

In [ ]:
# Change according to instruction above
analytical_solution_tag = "-ana_mexi_hat"
generate_config = generateConfig_squareMexiHat
generate_mesh = generateMesh_squareMexiHat

# analytical_solution_tag = "-ana_square_top"
# generate_config = generateConfig_squareTop
# generate_mesh = generateMesh_squareTop

# analytical_solution_tag = "-ana_square_sincos"
# generate_config = generateConfig_squareSinCos
# generate_mesh = generateMesh_squareSinCos

## Analysis setup

In [ ]:
# which executable?

exe = hdiv_diffusion
sumanalys = "sumanalys.csv"
ana_name = "ana_square_mexi_mixed_order"

ana_compare_exe = [hdiv_diffusion, classic_diffusion]
ana_compare_name = ["ana_square_mexi_mixed", "ana_square_mexi_classic"]
# ana_compare_name = ["ana_square_mexi_mixed"]
ana_compare_sum = ["sumanalys.csv", "FEM_errors.csv"]

# Convergence analysis parameters
order_list = [1, 2, 3] # approximation order p
elem_size_list = [0.05, 0.02, 0.01, 0.005] # element size h
# order_list = [1, 2, 3] # approximation order p
# elem_size_list = [0.5, 0.2, 0.1] # element size h
params.triangle_mesh = False
params.nproc = 10 # number of processors

run_test = True
run_analysis = True

naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
          "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter", "errorIndicatorGrad", "errorIndicatorDiv", "jumpL2", "jumpHdiv", "eleNum"]
# naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
#           "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter"]

error_name_list = ["L2norm", "H1seminorm", "fluxErr"]
error_label_list = [(r'Global error $L^2$-norm'),
               (r'Global error $H^1$-seminorm'), (r'Global Flux error')]

In [ ]:
params.conductivity = 1.0 # linear conductivity
params.element_size = 0.1 # element size in the regular mesh
params.order = 2 # approximation order for displacements

# params.triangle_mesh = False # use triangular mesh

# Pre-processing parameters
params.mesh_file = "square_mexi"
params.length_x = 1
params.length_y = 1
params.length_z = 0
params.show_mesh = True


# solution parameters
params.log_file = "log" # log file name 


## Run test

In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

In [ ]:
# Testing mesh generation
if run_test:
    params.show_mesh = True
    generate_config(params)
    generate_mesh(params)

In [ ]:
# Testing running analysis
if run_test:
    params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
    !{mofem_part} -my_file {params.mesh_file + ".h5m"} -nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
    !mpirun -np {params.nproc} {exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag}

    !convert.py out*


In [ ]:
if run_test:
    # params.show_file = "out_ori_result"
    params.show_file = "out_result"
    params.show_field = "P_reference"
    params.show_edges = True
    params.p_save = "run_test_p.pdf"
    show_results(params)

In [ ]:
if run_test:
    params.show_file = "out_error"
    params.show_field = "ERROR_INDICATOR_DIV"
    params.show_edges = True
    params.p_save = "run_test_err_ind_grad.pdf"
    show_results(params)

## Comparison between standard and mixed

In [ ]:
if run_analysis:    
    for i in range(len(ana_compare_name)):
        !rm {ana_compare_sum[i]}
        !rm ./output_files/out*
        for elem_size in elem_size_list:
            params.element_size = elem_size
            params.show_mesh = False
            generate_mesh(params)
            params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
            !{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
            for order in order_list:
                params.order = order
                !mpirun -np {params.nproc} {ana_compare_exe[i]} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag}
        !mv {ana_compare_sum[i]} {ana_compare_name[i]}.csv
    


In [ ]:
exe = hdiv_diffusion
sumanalys = "sumanalys.csv"
ana_name = "ana_square_mexi_mixed_order"

if run_analysis:    
    !rm sumanalys
    !rm ./output_files/out*
    for elem_size in elem_size_list:
        params.element_size = elem_size
        params.show_mesh = False
        generate_mesh(params)
        params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
        !{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
        for order in order_list:
            params.order = order
            !mpirun -np {params.nproc} {ana_compare_exe[i]} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag}
    !mv {ana_compare_sum[i]} {ana_compare_name[i]}.csv

### Load analysis

In [ ]:
print(naming)

In [ ]:
error_name_list = []
error_label_list = []

# error_name_list.append("errorEstimator")
# error_label_list.append(r'Global error estimator')
# error_name_list.append("errorIndicatorGrad")
# error_label_list.append(r'Global error indicator grad')
# error_name_list.append("errorIndicatorDiv")
# error_label_list.append(r'Global error indicator div')
# error_name_list.append("jumpL2")
# error_label_list.append(r'Global jump L2')
# error_name_list.append("jumpHdiv")
# error_label_list.append(r'Global jump Hdiv')

error_name_list.append("L2norm")
error_label_list.append(r'Global error $L^2$-norm')
error_name_list.append("H1seminorm")
error_label_list.append(r'Global error $H^1$-seminorm')
error_name_list.append("fluxErr")
error_label_list.append(r'Global Flux error')


In [ ]:
pre_name = "c3_mixed_hat_"

mixed_ana = Analysis(ana_compare_name[0], naming, order_list, error_name_list, error_label_list, pre_name, elem_size_list,  marker='x', linestyle='--', plot_gradients=True, label="Mixed")
classic_ana = Analysis(ana_compare_name[1], naming, order_list, error_name_list, error_label_list, pre_name, elem_size_list,  marker='o', linestyle='-', plot_gradients=False, label="Classic")

### Plot results

In [ ]:
mixed_ana.plot_both_analyses_by_elem_size([classic_ana])
# mixed_ana.plot_both_analyses_by_elem_size([])

In [ ]:
mixed_ana.plot_both_analyses_by_gaussnum([classic_ana])
# mixed_ana.plot_both_analyses_by_gaussnum([])
